**Task**

For this assignment, you will continue revisiting your first project to predict supermarket sales. You will be applying everything you've learned thus far this week to explain how your models made their predictions.

**Project 1 Revisited - Part 2: Global Explanations**

**Load Your Saved Joblib File**

Create a new notebook for this assignment in your project 1 repository. Name the notebook "Explaining Models with Shap.ipynb"

In your notebook, load the contents of your "best-models.joblib" file into a variable called "loaded_joblib."

Save each object from the loaded_joblib dictionary as a separate variable in your notebook. (e.g. "X_train = loaded_joblib['X_train'])

**Explain your tree-based model with shap:**

Create an X_shap and y_shap variable from your training data (use shap.sample as needed).

Create a model explainer,
Calculate the shap values for your model.

Create a summary plot - with plot_type='bar':
In a Markdown cell below, display your saved feature importance image (that you used in your README) and compare the most important features according to SHAP vs. your original feature importances.

Are they the same features in both? If not, what's different?

Save your bar summary plot figure as a .png file inside your repository (you will need this for the final piece of this assignment - Update Your README).

Create a second summary plot - with plot_type='dot'
In a markdown cell, interpret the top 3 most important features and how they influence your model's predictions.
Save your figure as a .png file inside your repository (you will need this for the final piece of this assignment - Update Your README).

**Update your README.**

Insert your exported figures and corresponding interpretations from above into your README file. You should have:

a summary plot - bar version

Your comparison of most important features found by shap vs feature importance.

a summary plot - dot version

Your interpretation of the top 3 most important features and how they influence your model's predictions.

Submit the link to your project repository as your final submission for this assignment.





In [1]:
##Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as miss

##Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

##Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

##Setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
##Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

**Load Your Saved Joblib File**

In [10]:
import joblib, os

NameError: name 'export' is not defined

In [8]:
# Try Loading again to make sure it works.

loaded = joblib.load(fname)
loaded.keys()

NameError: name 'fname' is not defined

In [7]:
# load data from part 1 of 'Project 1 Revisited'
loaded_joblib = joblib.load('best-models.joblib')
loaded_joblib.keys()

FileNotFoundError: [Errno 2] No such file or directory: 'best-models.joblib'